In [ ]:
from keras.models import load_model
from keras_bert import get_custom_objects
import os.path
import ktrain.text.preprocessor as tpp
import pickle

In [ ]:
bert_model = load_model('omdena_wri/static/coref_bert.hdf5', custom_objects=get_custom_objects())
preproc_type = tpp.TEXT_PREPROCESSORS.get('bert', None)
preproc = preproc_type(maxlen=350,
                        max_features=35000,
                        classes = ['pos','neg'],
                        lang='en', ngram_range=1)

In [ ]:
def is_land_conflict(text):
    """Returns True if text is environmental conflict related, otherwise returns False

        Parameters:
        text (str): news article as a string object

        Returns:
        is_conflict (bool): True if text is an env/land conflict; false otherwise.

    """

    # preprocess the text
    (text_processed,  _) = preproc.preprocess_train([text], verbose=1)

    # return model's prediction 
    return bool(bert_model.predict(text_processed).argmax(axis=1) )